In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

C:\Users\dk\AppData\Roaming\Python\Python36\site-packages\paramiko\transport.py:33: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend
c:\Users\dk\anaconda3\envs\py362\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: azureml.core: AzureML support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use AzureML will continue to work without modification, but Python 3.6 users will no longer get access to the latest AzureML features and bugfixes. We recommend that you upgrade to Python 3.7 or newer. To disable SDK V1 deprecation warning set the environment variable AZUREML_DEPRECATE_WARNING to 'False'
  """Entry point for launching an IPython kernel.


Workspace name: quick-starts-ws-242683
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-242683


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "project1-cc" # no underscore allowed!

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found the cluster, you can use it.")
except:
    print("Creating a new compute cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4, vm_priority= 'lowpriority')
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)

Found the cluster, you can use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler ### YOUR CODE HERE ###
ps = RandomParameterSampling( 
    {
        "--C" : uniform(0.1, 10),
        "--max_iter" : choice(15, 25, 50, 75, 100)
    }
    )

# Specify a Policy ### YOUR CODE HERE ###
policy = BanditPolicy(
    slack_factor=0.1, 
    evaluation_interval=2,
    delay_evaluation=10
    )

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job

src = ScriptRunConfig(
    source_directory='.',
    command=['python', 'train.py'],
    compute_target=cluster_name,
    environment=sklearn_env
    )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy. ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    run_config=src, 
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
    )

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config) 

# show run details with the widget
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Current provisioning state of AmlCompute is "Deleting"



In [27]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
display(best_run.get_metrics())

{'Regularization Strength:': 1.0,
 'Max iterations:': 100,
 'Accuracy': 0.9108883575501195}

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
df = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.core import Dataset, Datastore

# Split data into train and test sets. Set random state to ensure that we have the same partitions for HyperDrive and AutoML
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42) # classes are imbalance, stratification would be better

train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

# Register the train and test sets
datastore_ws = Datastore.get(ws, 'workspaceblobstore')
train_ds = Dataset.Tabular.register_pandas_dataframe(train_df, datastore_ws, "train data", show_progress=True)
test_ds = Dataset.Tabular.register_pandas_dataframe(test_df, datastore_ws, "test data", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/2cfe12e0-172a-4bdf-b95a-9f2a25276618/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    blocked_models = ["LogisticRegression"],
    primary_metric = "accuracy",
    training_data=train_ds,
    validation_data=test_ds,
    label_column_name='y',
    compute_target=cluster_name)

In [16]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on project1-cc with default configuration
Running on remote compute: project1-cc


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_6a328790-8dee-4b43-bbc4-619eb53ef47f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in th

In [21]:
# Retrieve and save your best automl model.
automl_best_run, automl_best_model = automl_run.get_output()
display(automl_best_run)

# Step 3: Save the best AutoML model
joblib.dump(automl_best_model, 'automl_model.pkl')

Package:azureml-automl-runtime, training version:1.48.0.post2, current version:1.47.0.post1
Package:azureml-core, training version:1.48.0, current version:1.47.0
Package:azureml-dataprep, training version:4.8.6, current version:4.5.7
Package:azureml-dataprep-rslex, training version:2.15.2, current version:2.11.4
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.47.0
Package:azureml-interpret, training version:1.48.0, current version:1.47.0
Package:azureml-mlflow, training version:1.48.0, current version:1.47.0
Package:azureml-telemetry, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-runtime, training version:1.48.0.post2, current version:1.47.0.post1
Package:azureml-train-core, training version:1.48.0, current version:1.47.0
Package:azureml-train-restclients-hyperdrive, training version:1.48.0, current version:1.47.0
Package:azureml-training-tabul

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_6a328790-8dee-4b43-bbc4-619eb53ef47f_40,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


['automl_model.pkl']

In [23]:
# delete cluster
compute_target.delete()